In [141]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
    break
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import json 
import cv2

import pandas as pd 

In [142]:
img=cv2.imread("../input/wider-face-recognization/WIDER_train/WIDER_train/images/0--Parade/0_Parade_marchingband_1_799.jpg")

In [143]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [144]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [145]:
d=open('../input/wider-face-recognization/wider_face_split/wider_face_split/wider_face_train_bbx_gt.txt','r').read()
d= d.split('\n')
files= {}
x=0
cur_index=0
while x<(len(d)-1) :    
    if d[x].find('.jpg')>-1:
        files[d[x]]={}
        files[d[x]]['num_faces']=d[x+1]
        files[d[x]]['bbox']=[]

        #print(d[x])
        cur_index=x+2
        while True:
            try:
                if d[cur_index].find('.jpg')>-1:
                    break
            except:
                print(f"Catch {cur_index} completed")
                break
            try:
                files[d[x]]['bbox'].append([int(x) for x in  d[cur_index].strip().split(' ') ] )
            except:
                print(f"This threw error {cur_index} {d[cur_index]}")
                pass
            cur_index+=1
        x=cur_index

    

In [ ]:
train_df=pd.DataFrame(files.items())
train_df[train_df.iloc[:,1].apply(lambda x : x['num_faces'])=='1']

In [ ]:
def transformCoordinates(bb):
    bb=bb['bbox']
    new_bb=[]
    for x in bb:
        new_bb.append(np.array([x[0],x[1],x[0]+x[2],x[1]+x[3]]))
    return np.array(new_bb)

In [ ]:
train_df

In [ ]:
train_df['bb_trans']=train_df.iloc[:,1].apply(transformCoordinates)

In [ ]:
train_df

In [ ]:
ind=12875
img=cv2.imread(f'../input/wider-face-recognization/WIDER_train/WIDER_train/images/{train_df.iloc[ind,0]}',cv2.COLOR_BGR2RGB)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb )

for x in train_df.loc[ind,'bb_trans']:
    print(x)
    cv2.rectangle(img_rgb,(x[0],x[1]),(x[2],x[3]),(40,125,200),2)
    print(x[:4])
plt.figure(figsize = (14.8,10.8))

plt.imshow(img_rgb)

In [ ]:
import numpy as np
def IoU(box, boxes):
    """Compute IoU between detect box and gt boxes
    Parameters:
    ----------
    box: numpy array , shape (5, ): x1, y1, x2, y2, score 
        input box
    boxes: numpy array, shape (n, 4): x1, y1, x2, y2
        input ground truth boxes
    Returns:
    -------
    ovr: numpy.array, shape (n, )         IoU
    """
    # box = (x1, y1, x2, y2)
    
    box_area = (box[2] - box[0] + 1) * (box[3] - box[1] + 1)
    area = (boxes[:, 2] - boxes[:, 0] + 1) * (boxes[:, 3] - boxes[:, 1] + 1)
    # abtain the offset of the interception of union between crop_box and gt_box
    xx1 = np.maximum(box[0], boxes[:, 0])
    yy1 = np.maximum(box[1], boxes[:, 1])
    xx2 = np.minimum(box[2], boxes[:, 2])
    yy2 = np.minimum(box[3], boxes[:, 3])
    w = np.maximum(0, xx2 - xx1 + 1)
    h = np.maximum(0, yy2 - yy1 + 1)
    inter = w * h
    ovr = inter / (box_area + area - inter)
    return ovr

In [ ]:
train_df.loc[856,'bb_trans']

In [ ]:
IoU([490, 260, 500, 362],train_df.loc[856,'bb_trans'])

In [ ]:
import torch.nn as nn

In [ ]:
class PnetMy(nn.Module):
    def __init__(self):
        super().__init__()
        self.cov1 = nn.Conv2d( in_channels =3,out_channels= 10,kernel_size =(3,3),stride= 1)
        self.prelu1 = nn.PReLU(10)
        self.maxPool1 = nn.MaxPool2d(kernel_size=(3,3) ,stride=2,ceil_mode=True)
        self.cov2=nn.Conv2d( in_channels= 10,out_channels= 16,kernel_size= (3,3),stride= 1)
        self.prelu2 = nn.PReLU(16)
        self.cov3=nn.Conv2d( in_channels= 16,out_channels= 32,kernel_size= (3,3),stride= 1)
        self.prelu3 = nn.PReLU(32)
        self.face_detector=nn.Conv2d( in_channels= 32,out_channels= 1,kernel_size= (1,1),stride= 1)
        self.softmax = nn.Softmax(dim=1)
        self.bb_regression=nn.Conv2d( in_channels= 32,out_channels= 4,kernel_size= (1,1),stride= 1)
        self.facial_landmark=nn.Conv2d( in_channels= 32,out_channels= 10,kernel_size= (1,1),stride= 1)
    def forward(self,x):
        x=self.cov1(x)
        x=self.prelu1(x)
        #print(f"Shape should be n*10*10*10 {x.shape}")
        x=self.maxPool1(x)
        #print(f"Shape after MaxPool should be n*10*5*5 {x.shape}")
        x=self.cov2(x)
        x=self.prelu2(x)
        #print(f"Shape should be n*16*3*3 {x.shape}")
        x=self.cov3(x)
        x=self.prelu3(x)
        #print(f"Shape should be n*32*1*1 {x.shape}")
        is_face=self.face_detector(x)
        is_face=is_face.view((-1,1))
        #is_face=self.softmax(is_face)
        bb_output=self.bb_regression(x)
        bb_output=bb_output.view((-1,4))
        return is_face,bb_output

In [ ]:
import torch.nn.functional as F

In [ ]:
import torch

In [ ]:
mypnet=PnetMy()

In [ ]:
dim_input = torch.randn( 2,3, 500,500)
is_face,bb=mypnet(dim_input)
check1= nn.Conv2d(in_channels=3,out_channels=4,kernel_size=12,stride=2,padding=0)(dim_input)
check1.shape,bb.shape , is_face.shape

In [ ]:
!mkdir ./pnet
!mkdir ./pnet/train


!mkdir ./pnet/train/neg
!mkdir ./pnet/train/pos
!rm ./pnet/train/neg -r
!rm ./pnet/train/pos -r

!mkdir ./pnet/train/neg
!mkdir ./pnet/train/pos

In [ ]:
!tree 

In [ ]:
from tqdm.notebook import tqdm
import cv2
import uuid
import time
p_idx=0
n_idx=0
d_idx=0
idx=1
box_idx=0
neg_directory='./pnet/train/neg'
pos_directory='./pnet/train/pos'
labels={}
orig=time.time()
tick=orig
for x in tqdm(train_df.iterrows(),total=train_df.shape[0]):
    try:
        #print(x[1][0])
        file_path=f'../input/wider-face-recognization/WIDER_train/WIDER_train/images/{x[1][0]}'
        bbox=x[1]['bb_trans']
        #print(bbox)
        img=cv2.imread(file_path,cv2.COLOR_BGR2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, channel = img.shape
        #print(img.shape)
        idx+=1
        if idx%250 ==0 :
            last_tick=tick
            tick=time.time()

            print(f"Completed {idx} images in {tick-last_tick}   total_time_passed {tick-orig}  average : {(tick-last_tick)/250}")
            print("%s images done, pos: %s part: %s neg: %s" % (idx, p_idx, d_idx, n_idx))

        neg_num = 0
        max_retires=25
        current_retry=0
        while neg_num < 10 and current_retry< max_retires:
            current_retry+=1
            size = np.random.randint(12, min(width, height) / 2)
            nx = np.random.randint(0, width - size)
            ny = np.random.randint(0, height - size)
            crop_box = np.array([nx, ny, nx + size, ny + size])

            Iou = IoU(crop_box, bbox)

            cropped_im = img[ny: ny + size, nx: nx + size, :]
            resized_im = cv2.resize(cropped_im, (12, 12), interpolation=cv2.INTER_LINEAR)
            filename=str(uuid.uuid4())
            if np.max(Iou) < 0.3:
                # Iou with all gts must below 0.3
                save_file=f"{neg_directory}/{filename}_0_nonoverlap.jpg"
                labels[f"{filename}_0_nonoverlap.jpg"]={'label':0,'bb':[]}
                cv2.imwrite(save_file, resized_im)
                n_idx += 1
                neg_num += 1
        for box in bbox:
            # box (x_left, y_top, x_right, y_bottom)
            x1, y1, x2, y2 = box
            # w = x2 - x1 + 1
            # h = y2 - y1 + 1
            w = x2 - x1 + 1
            h = y2 - y1 + 1

            # ignore small faces
            # in case the ground truth boxes of small faces are not accurate
            if max(w, h) < 40 or x1 < 0 or y1 < 0:
                continue

                    # generate negative examples that have overlap with gt
            for i in range(5):
                size = np.random.randint(12, min(width, height) / 2)
                # delta_x and delta_y are offsets of (x1, y1)

                delta_x = np.random.randint(max(-size, -x1), w)
                delta_y = np.random.randint(max(-size, -y1), h)
                nx1 = max(0, x1 + delta_x)
                ny1 = max(0, y1 + delta_y)

                if nx1 + size > width or ny1 + size > height:
                    continue
                crop_box = np.array([nx1, ny1, nx1 + size, ny1 + size])
                Iou = IoU(crop_box, bbox)

                cropped_im = img[ny1: ny1 + size, nx1: nx1 + size, :]
                resized_im = cv2.resize(cropped_im, (12, 12), interpolation=cv2.INTER_LINEAR)
                filename=str(uuid.uuid4())
                if np.max(Iou) < 0.3:
                    # Iou with all gts must below 0.3
                    save_file=f"{neg_directory}/{filename}_0_overlap.jpg"
                    #f2.write(save_file + ' 0\n')
                    labels[f"{filename}_0_overlap.jpg"]={'label':0,'bb':[]}
                    cv2.imwrite(save_file, resized_im)
                    n_idx += 1
            for i in range(8):
                size = np.random.randint(int(min(w, h) * 0.8), np.ceil(1.25 * max(w, h)))

                # delta here is the offset of box center
                delta_x = np.random.randint(-w * 0.15, w * 0.15)
                delta_y = np.random.randint(-h * 0.15, h * 0.15)

                nx1 = max(x1 + w / 2 + delta_x - size / 2, 0)
                ny1 = max(y1 + h / 2 + delta_y - size / 2, 0)
                nx2 = nx1 + size
                ny2 = ny1 + size

                if nx2 > width or ny2 > height:
                    continue
                crop_box = np.array([nx1, ny1, nx2, ny2])

                offset_x1 = (x1 - nx1) / float(size)
                offset_y1 = (y1 - ny1) / float(size)
                offset_x2 = (x2 - nx2) / float(size)
                offset_y2 = (y2 - ny2) / float(size)

                cropped_im = img[int(ny1): int(ny2), int(nx1): int(nx2), :]
                resized_im = cv2.resize(cropped_im, (12, 12), interpolation=cv2.INTER_LINEAR)

                box_ = box.reshape(1, -1)
                filename=str(uuid.uuid4())
                if IoU(crop_box, box_) >= 0.70:

                    save_file=f"{pos_directory}/{filename}_1_pos.jpg"
                    labels[f"{filename}_1_pos.jpg"]={'label':1,'bb':     [offset_x1, offset_y1, offset_x2, offset_y2]}
                    cv2.imwrite(save_file, resized_im)
                    p_idx += 1
                elif IoU(crop_box, box_) >= 0.4:
                    save_file=f"{pos_directory}/{filename}_1_part.jpg"
                    labels[f"{filename}_1_part.jpg"]={'label':-1,'bb':     [offset_x1, offset_y1, offset_x2, offset_y2]}
                    cv2.imwrite(save_file, resized_im)
                    d_idx += 1

            box_idx += 1
    except Exception as e:
        #raise e 
        print("Encountered exception ")
        pass
        
        
import pickle as pkl

#pkl.dump(labels,open("annotations.pkl",'wb'))
import json 
json.dump(labels, open("annotations.json","w"), indent = 1)


In [ ]:
def getfilename(x,base_directory="./pnet/train"):
    neg_directory="neg"
    pos_directory="pos"
    if int(x.split('_')[-2])==0:
        return f"{base_directory}/{neg_directory}/{x}"
    else:
        return f"{base_directory}/{pos_directory}/{x}"
        

In [ ]:
import torch

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
class PNetDataset(Dataset):
    """Face Landmarks dataset."""
    def __init__(self, data,file_ids, directory, transform=None):
        self.all_files=data
        self.file_ids=file_ids
        self.directory=directory
        self.transform=transform
    def __len__(self):
        return len(self.file_ids)

    def __getitem__(self, idx):
        
        img_name = getfilename(self.file_ids[idx])
        #print(f"{img_name}\n")
        labels= self.all_files[self.file_ids[idx]]['label']
        if labels!=0:
            labels=1
        bb = np.array(self.all_files[self.file_ids[idx]]['bb'],dtype=float)
        if len(bb)==0:
            bb=np.array([0,0,0,0],dtype=float)
        
        image=cv2.imread(img_name)
        image=image/255
        image=image.astype(np.float32)
        
        
        return {'image':torch.from_numpy(image).permute(2, 0, 1),'label':float(labels),'bb':torch.from_numpy(bb)}    
        

In [ ]:
myPnetDataset=PNetDataset(data=labels,file_ids= list(labels.keys()),directory='./pnet/train')
#trainDataloader = DataLoader(myPnetDataset,batch_size=32 ,num_workers=3, )

In [ ]:
mypnet=PnetMy()
mypnet.train()


In [ ]:
loss1 =torch.nn.BCEWithLogitsLoss()
loss2 =torch.nn.MSELoss()
optimizer = torch.optim.Adam(mypnet.parameters(), lr=0.02)
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)
mypnet.to(device)
batch_size=2048
trainDataloader = DataLoader(myPnetDataset,batch_size=batch_size ,num_workers=4, )
for epoch in range(0,5):
    cls_loss_sum=0
    bb_loss_sum=0
    for batch_id, data in enumerate(trainDataloader):
        images=data['image'].float().to(device)
        gt_bbox=data['bb'].float().to(device)
        gt_isface=data['label'].float().to(device)
        is_face, bb= mypnet(images)
        cls_loss=loss1(is_face.float(),gt_isface.reshape(gt_isface.shape[0],1).float())
        bb_loss=loss2(bb.float(), gt_bbox.float())
        cls_loss_sum=cls_loss+cls_loss_sum
        bb_loss_sum=bb_loss+bb_loss_sum
        
        loss_all= cls_loss+bb_loss
        optimizer.zero_grad()
        loss_all.backward()
        optimizer.step()
        if batch_id%80==0:
            print(f"{batch_id} {bb_loss_sum/((batch_id+1)*1)} {cls_loss_sum/((batch_id+1)*1)}")


In [ ]:
for epoch in range(0,2):
    cls_loss_sum=0
    bb_loss_sum=0
    for batch_id, data in enumerate(trainDataloader):
        images=data['image'].float().to(device)
        gt_bbox=data['bb'].float().to(device)
        gt_isface=data['label'].float().to(device)
        is_face, bb= mypnet(images)
        cls_loss=loss1(is_face.float(),gt_isface.reshape(gt_isface.shape[0],1).float())
        bb_loss=loss2(bb.float(), gt_bbox.float())
        cls_loss_sum=cls_loss+cls_loss_sum
        bb_loss_sum=bb_loss+bb_loss_sum
        
        loss_all= cls_loss+bb_loss
        optimizer.zero_grad()
        loss_all.backward()
        optimizer.step()
        if batch_id%20==0:
            print(f"{batch_id} {bb_loss_sum/((batch_id+1))} {cls_loss_sum/((batch_id+1))}")


In [ ]:
### scoring on the training set 

In [ ]:
scores=0
from sklearn.metrics import roc_auc_score
for index , K in enumerate(trainDataloader) :
    is_face, bb= mypnet(K['image'].to(device))
    scores+=roc_auc_score(K['label'],nn.Sigmoid()(is_face).cpu().detach().numpy())
    if index%50==0:
        print(f"Target Label % {K['label'].sum()/K['label'].shape[0]} ,Batch ROC-AUC {(roc_auc_score(K['label'],nn.Sigmoid()(is_face).cpu().detach().numpy()) )} ,Overall ROC-AUC {scores/(index+1)}" )

In [ ]:
torch.save(mypnet.state_dict(), os.path.join('./',"pnet_epoch_.pt"))
torch.save(mypnet, os.path.join('./',"pnet_epoch_model.pkl"))

In [ ]:

def resize_image( img, scale):
    """
        resize image and transform dimention to [batchsize, channel, height, width]
    Parameters:
    ----------
        img: numpy array , height x width x channel
            input image, channels in BGR order here
        scale: float number
            scale factor of resize operation
    Returns:
    -------
        transformed image tensor , 1 x channel x height x width
    """
    height, width, channels = img.shape
    new_height = int(height * scale)     # resized new height
    new_width = int(width * scale)       # resized new width
    new_dim = (new_width, new_height)
    img_resized = cv2.resize(img, new_dim, interpolation=cv2.INTER_LINEAR)      # resized image
    return img_resized

In [ ]:
def changetotensor(img):
    """
    convert H,W,C numpy image to C,H,W tensor
    """
    #img=cv2.resize(img,(12,12))
    ten= torch.from_numpy(img).float().permute(2, 0, 1)
    output=torch.empty(size=(1,3,ten.shape[1],ten.shape[2]))
    output[0]=ten/255
    print(output.shape)
    return output

In [ ]:
list(mypnet.children())

In [ ]:
   
import torchvision.transforms as transforms
import torch
from torch.autograd.variable import Variable
import numpy as np

transform = transforms.ToTensor()

In [ ]:
def myOutputBb(mypnet,k):
    return mypnet.bb_regression(mypnet.prelu3(mypnet.cov3(mypnet.prelu2(mypnet.cov2(mypnet.maxPool1(mypnet.prelu1((mypnet.cov1(k)))))))))
def myOutputcls(mypnet,k):
    return mypnet.face_detector(mypnet.prelu3(mypnet.cov3(mypnet.prelu2(mypnet.cov2(mypnet.maxPool1(mypnet.prelu1((mypnet.cov1(k)))))))))
    
     
    

In [ ]:
all_boxes = list()
i = 0
ind=12875
#ind=542
im=cv2.imread(f'../input/wider-face-recognization/WIDER_train/WIDER_train/images/{train_df.iloc[ind,0]}',cv2.COLOR_BGR2RGB)
im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
plt.imshow(im)
h, w, c = im.shape

net_size = 12

current_scale = float(net_size) / 350# find initial scale
# print('imgshape:{0}, current_scale:{1}'.format(im.shape, current_scale))
im_resized = resize_image(im, current_scale) # scale = 1.0
current_height, current_width, _ = im_resized.shape



In [ ]:
im_resized.shape

In [ ]:
plt.imshow(im)

In [ ]:
import math 
img=cv2.resize(im,(50,50)) #### only squares 
has_face,face_activated_bb= mypnet(changetotensor(img).to(device))
face_activated_bb.shape

re_im =int(math.sqrt(face_activated_bb.shape[0]))
has_face.shape,bb.shape
plt.imshow(F.sigmoid(has_face).view((re_im,re_im)).detach().cpu().numpy())
plt.show()
plt.imshow(img)


In [ ]:
import math 
img=cv2.resize(im,(150,250)) #### any resize  
has_face=myOutputcls(mypnet,changetotensor(img).to(device))
print(has_face.shape)
plt.imshow(F.sigmoid(has_face[0,0,:,:]).detach().cpu().numpy())
plt.show()
plt.imshow(img)


In [248]:
all_boxes = list()
i = 0
ind=12875
#ind=542
im=cv2.imread(f'../input/wider-face-recognization/WIDER_train/WIDER_train/images/{train_df.iloc[ind,0]}',cv2.COLOR_BGR2RGB)
im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
plt.imshow(im)
h, w, c = im.shape

net_size = 12

current_scale = float(net_size) / 350# find initial scale
# print('imgshape:{0}, current_scale:{1}'.format(im.shape, current_scale))
im_resized = resize_image(im, current_scale) # scale = 1.0
current_height, current_width, _ = im_resized.shape
k=changetotensor(im_resized)
cls_map, reg = myOutputcls(mypnet.to(device),k.to(device)),myOutputBb(mypnet.to(device),k.to(device))
cls_map=cls_map.permute(0,2,3,1)
plt.imshow(F.sigmoid(cls_map).detach().cpu().numpy()[0]>0.240)
plt.show()
plt.imshow(im_resized)
plt.show()

In [249]:
reg_bb=reg.permute(0,2,3,1)[0].detach().cpu().numpy()

In [250]:
reg_bb

In [251]:
lon=np.array(np.where(F.sigmoid(cls_map).detach().cpu().numpy()[0]>0.85))

In [252]:
lon

In [253]:
reg_bb.shape,current_scale

In [254]:
final_boxes=[]
for x in range(len(lon[0])):
    cell_pos= lon[:,x]
    coordinates=reg_bb[cell_pos[0],cell_pos[1],:]
    real_ord=np.array([(2*cell_pos[1])/current_scale ,(2*cell_pos[0])/current_scale ,(2*cell_pos[1] +12)/current_scale , (2*cell_pos[0] + 12)/current_scale],dtype=int)
    print(coordinates)
    print(real_ord)
    final_boxes.append(real_ord)
    cv2.rectangle(im,(real_ord[0],real_ord[1]),(real_ord[2],real_ord[3]),(40,125,200),2)
    plt.imshow(im)
    plt.title(f"From {x} Activation Map")
    plt.show()
    
    #input()
    

In [255]:
final_boxes=np.array(final_boxes)
final_scores=F.sigmoid(cls_map)[0][lon]

In [256]:
import numpy as np


def torch_nms(dets,scores, thresh, mode="Union"):
    """
    greedily select boxes with high confidence
    keep boxes overlap <= thresh
    rule out overlap > thresh
    :param dets: [[x1, y1, x2, y2 score]]
    :param thresh: retain overlap <= thresh
    :return: indexes to keep
    """
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores =scores

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]
    print(order)
    keep = []
    while order.size > 0:
        
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        print(i)
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        if mode == "Union":
            ovr = inter / (areas[i] + areas[order[1:]] - inter)
        elif mode == "Minimum":
            ovr = inter / np.minimum(areas[i], areas[order[1:]])
        print(ovr)
        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]
        print("Tjere")

    return keep

In [257]:
final_scores

In [275]:
to_keep_indexes=torch_nms(final_boxes,final_scores.detach().cpu().numpy(),0.4)

In [276]:
to_keep_indexes

In [277]:
im=cv2.imread(f'../input/wider-face-recognization/WIDER_train/WIDER_train/images/{train_df.iloc[ind,0]}',cv2.COLOR_BGR2RGB)
im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
for x in to_keep_indexes:
    print()
    cv2.rectangle(im,(final_boxes[x][0],final_boxes[x][1]),(final_boxes[x][2],final_boxes[x][3]),(100,5,200),2)
plt.imshow(im)
plt.title(f"From {x} Activation Map")
plt.show()

In [281]:
!zip -r -qq pnet_complete.zip ./pnet/*

In [282]:
!rm ./pnet/train/neg -r
!rm ./pnet/train/pos -r